In [ ]:
# === STEP 1: SETUP ===
from google.colab import drive
import os

# 1. Mount Google Drive (You will be asked to click a link and authorize)
print("📂 Connecting to Google Drive...")
drive.mount('/content/drive')

# 2. Install AI Libraries
print("⏳ Installing libraries...")
os.system('pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" --quiet')
os.system('pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes --quiet')
os.system('pip install langchain langchain-community langchain-huggingface chromadb flask pyngrok duckduckgo-search --quiet')

print("✅ Step 1 Complete: Drive mounted and libraries installed.")


In [ ]:
# === STEP 2: SMART DATASET LOADING ===
import json
import os

# 1. Define the path in Google Drive
dataset_path = "/content/drive/My Drive/my_training_data.jsonl"

# 2. Check if you already have a custom dataset
if os.path.exists(dataset_path):
    print(f"📂 Found custom dataset at: {dataset_path}")
    print("   Using your personal data for training.")

else:
    print("⚠️ No custom dataset found. Creating a default one for you...")

    # --- DEFAULT DATA (Generic Template) ---
    default_data = [
        {"instruction": "Who are you?", "output": "I am an AI assistant trained to help you."},
        {"instruction": "What can you do?", "output": "I can answer questions, write code, and assist with tasks."},
        {"instruction": "Explain AI.", "output": "AI stands for Artificial Intelligence, which enables computers to mimic human logic."},
        {"instruction": "Write a hello world in Python.", "output": "print('Hello World')"}
    ]

    # Save this default data to Drive so you can edit it later
    with open(dataset_path, 'w') as f:
        for entry in default_data:
            json.dump(entry, f)
            f.write('\n')

    print(f"✅ Created default dataset at: {dataset_path}")
    print("   (To use your own data, just edit this file in your Google Drive!)")

print("✅ Step 2 Complete: Dataset is ready.")

In [ ]:
# === STEP 3: TRAIN MODEL ===
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

# 1. Load Base Model
print("⏳ Loading Llama-3.1-8B Model...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

# 2. Add LoRA Adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

# 3. Define Format
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

def formatting_func(examples):
    texts = []
    for instruction, output in zip(examples["instruction"], examples["output"]):
        texts.append(alpaca_prompt.format(instruction, output) + tokenizer.eos_token)
    return texts

# 4. Load Dataset (FROM GOOGLE DRIVE NOW)
dataset_file = "/content/drive/My Drive/my_training_data.jsonl"
print(f"📂 Loading dataset from: {dataset_file}")

dataset = load_dataset("json", data_files=dataset_file, split="train")

# 5. Start Training
print("🚀 Starting Fine-Tuning...")
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 1,
    formatting_func = formatting_func,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)
trainer.train()

# 6. Save Locally (Temporary)
print("💾 Saving temporary model...")
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")
print("✅ Step 3 Complete: Training finished.")

In [ ]:
# === STEP 4: CREATE KNOWLEDGE BASE (RAG) ===
import os

# 1. Define the path in Google Drive
rag_path = "/content/drive/My Drive/my_knowledge.txt"

# 2. Check if it already exists
if os.path.exists(rag_path):
    print(f"📂 Found existing knowledge base at: {rag_path}")
    print("   (The bot will read facts from this file)")

else:
    print("⚠️ Creating a new knowledge base...")

    # --- DEFAULT FACTS (The "Cheat Sheet") ---
    # You can edit this file in your Google Drive anytime!
    knowledge_text = """
Identity:
- Name: CrownClown
- Creator: Joshua
- Creation Year: 2026
- Architecture: Llama 3.1 8B (Fine-tuned)

User Preferences:
- The user is a developer.
- Prefers code examples over long explanations.
"""

    with open(rag_path, "w") as f:
        f.write(knowledge_text)

    print(f"✅ Created default knowledge file at: {rag_path}")

print("✅ Step 4 Complete: Knowledge Base is ready.")

In [ ]:
# === STEP 5: PERMANENT SAVE TO DRIVE ===
import shutil
import os

# Define paths
source_folder = "lora_model"
destination_folder = "/content/drive/My Drive/CrownClown_Model"

print(f"💾 Copying '{source_folder}' to Google Drive...")

# 1. Clean up old version in Drive (if exists)
if os.path.exists(destination_folder):
    shutil.rmtree(destination_folder)

# 2. Copy the new model
shutil.copytree(source_folder, destination_folder)

print(f"✅ Step 5 Complete: Model successfully saved to {destination_folder}")

In [ ]:
# === STEP 5: FINAL SERVER (WITH MEMORY) ===
# 🔴 PASTE YOUR NGROK TOKEN BELOW 🔴
NGROK_AUTH_TOKEN = "387ajRwhcZVYCEMbS8PYwjrR04v_86Ws1H64EPTQx7c5bM25F"

import os
import torch
from flask import Flask, request, jsonify
from pyngrok import ngrok, conf
from unsloth import FastLanguageModel
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from duckduckgo_search import DDGS

# 1. Load Model
drive_model_path = "/content/drive/My Drive/CrownClown_Model"
print(f"⏳ Loading CrownClown from: {drive_model_path}...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = drive_model_path,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

# 2. Setup RAG
if not os.path.exists("my_knowledge.txt"):
    with open("my_knowledge.txt", "w") as f: f.write("Identity: CrownClown AI.")
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

def get_rag_context(query):
    if not os.path.exists("my_knowledge.txt"): return ""
    with open("my_knowledge.txt", "r") as f: doc_text = f.read()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = [Document(page_content=x) for x in text_splitter.split_text(doc_text)]
    if not docs: return ""
    db = Chroma.from_documents(docs, embedding_model)
    results = db.similarity_search(query, k=1)
    return results[0].page_content if results else ""

def search_web(query):
    try:
        results = DDGS().text(query, max_results=3)
        return "\n".join([f"- {r['title']}: {r['body']}" for r in results]) if results else ""
    except: return ""

app = Flask(__name__)

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    user_message = data.get("message", "")
    # FRONTEND sends the history now!
    chat_history = data.get("history", [])
    use_web = data.get("use_web", False)

    # 1. Get Context
    rag_context = get_rag_context(user_message)
    web_context = ""
    if use_web or any(k in user_message.lower() for k in ["plan", "price", "news", "trip", "weather"]):
        web_context = search_web(user_message)

    # 2. Format History from the Frontend
    # (The frontend sends ["User: Hi", "Bot: Hello"], we join it)
    history_text = "\n".join(chat_history[-6:])

    # 3. Construct Prompt with History
    # We tell it: "Here is what we said before. Continue the conversation."
    input_text = f"""You are CrownClown.
Context: {rag_context}
Web Info: {web_context}

PREVIOUS CHAT:
{history_text}

CURRENT USER QUESTION: {user_message}
System Note: Do not introduce yourself if you already did in the previous chat. Answer directly.
"""

    prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{input_text}

### Response:
"""

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        use_cache=True,
        stop_strings=["### Instruction"],
        tokenizer=tokenizer
    )

    response = tokenizer.batch_decode(outputs)[0]
    final_answer = response.split("### Response:")[-1].split("### Instruction")[0].replace("<|eot_id|>", "").strip()
    is_code = "```" in final_answer

    # No longer saving to internal memory (HISTORY) as it's stateless

    return jsonify({"response": final_answer, "is_code": is_code})

# Run
conf.get_default().auth_token = NGROK_AUTH_TOKEN
ngrok.kill()
public_url = ngrok.connect(5000).public_url
print(f"\n🚀 SMART SERVER LIVE: {public_url}")
app.run(port=5000)